In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split,KFold,cross_val_score,GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
data_train = pd.read_csv('Data.csv')
data_test = pd.read_csv('New_Data.csv')

In [3]:
data_train.head(1)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y


In [4]:
data_train.shape

(614, 13)

In [5]:
data_test

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban
...,...,...,...,...,...,...,...,...,...,...,...,...
362,LP002971,Male,Yes,3+,Not Graduate,Yes,4009,1777,113.0,360.0,1.0,Urban
363,LP002975,Male,Yes,0,Graduate,No,4158,709,115.0,360.0,1.0,Urban
364,LP002980,Male,No,0,Graduate,No,3250,1993,126.0,360.0,NaN,Semiurban
365,LP002986,Male,Yes,0,Graduate,No,5000,2393,158.0,360.0,1.0,Rural


In [6]:
data_train.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,614.000000,614.000000,592.000000,600.00000,564.000000
mean,5403.459283,1621.245798,146.412162,342.00000,0.842199
std,6109.041673,2926.248369,85.587325,65.12041,0.364878
min,150.000000,0.000000,9.000000,12.00000,0.000000
25%,2877.500000,0.000000,100.000000,360.00000,1.000000
50%,3812.500000,1188.500000,128.000000,360.00000,1.000000
75%,5795.000000,2297.250000,168.000000,360.00000,1.000000
max,81000.000000,41667.000000,700.000000,480.00000,1.000000


In [7]:
data_train.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [8]:
data_train['Credit_History'].value_counts()

1.0    475
0.0     89
Name: Credit_History, dtype: int64

In [9]:
def value_counts():
    null=data_train.isnull().sum()
    G_V=data_train['Gender'].value_counts()
    M_V=data_train['Married'].value_counts()
    SE_V=data_train['Self_Employed'].value_counts()
    CH_V=data_train['Credit_History'].value_counts()
    return null,G_V,M_V,SE_V,CH_V

In [10]:
value_counts()

(Loan_ID               0
 Gender               13
 Married               3
 Dependents           15
 Education             0
 Self_Employed        32
 ApplicantIncome       0
 CoapplicantIncome     0
 LoanAmount           22
 Loan_Amount_Term     14
 Credit_History       50
 Property_Area         0
 Loan_Status           0
 dtype: int64, Male      489
 Female    112
 Name: Gender, dtype: int64, Yes    398
 No     213
 Name: Married, dtype: int64, No     500
 Yes     82
 Name: Self_Employed, dtype: int64, 1.0    475
 0.0     89
 Name: Credit_History, dtype: int64)

In [11]:
def Preprocess():
    global combined
    train = pd.read_csv('Data.csv')
    test = pd.read_csv('New_Data.csv')
    targets = train.Loan_Status
    train.drop('Loan_Status', 1, inplace=True)
    combined = train.append(test)
    combined.reset_index(inplace=True)
    combined.drop(['index', 'Loan_ID'], inplace=True, axis=1)
    combined['Gender'].fillna('Male', inplace=True)
    combined['Married'].fillna('Yes', inplace=True)
    combined['Self_Employed'].fillna('No', inplace=True)
    combined['LoanAmount'].fillna(combined['LoanAmount'].median(), inplace=True)
    combined['Credit_History'].fillna(2, inplace=True) #For unknown credit card history consider 2
    combined['Gender'] = combined['Gender'].map({'Male':1,'Female':0})
    combined['Married'] = combined['Married'].map({'Yes':1,'No':0})
    combined['Singleton'] = combined['Dependents'].map(lambda d: 1 if d=='1' else 0)
    combined['Small_Family'] = combined['Dependents'].map(lambda d: 1 if d=='2' else 0)
    combined['Large_Family'] = combined['Dependents'].map(lambda d: 1 if d=='3+' else 0)
    combined.drop(['Dependents'], axis=1, inplace=True)
    combined['Education'] = combined['Education'].map({'Graduate':1,'Not Graduate':0})
    combined['Self_Employed'] = combined['Self_Employed'].map({'Yes':1,'No':0})
    combined['Total_Income'] = combined['ApplicantIncome'] + combined['CoapplicantIncome']
    combined.drop(['ApplicantIncome','CoapplicantIncome'], axis=1, inplace=True)
    combined['Debt_Income_Ratio'] = combined['Total_Income'] / combined['LoanAmount']
    combined['Very_Short_Term'] = combined['Loan_Amount_Term'].map(lambda t: 1 if t<=60 else 0)
    combined['Short_Term'] = combined['Loan_Amount_Term'].map(lambda t: 1 if t>60 and t<180 else 0)
    combined['Long_Term'] = combined['Loan_Amount_Term'].map(lambda t: 1 if t>=180 and t<=300  else 0)
    combined['Very_Long_Term'] = combined['Loan_Amount_Term'].map(lambda t: 1 if t>300 else 0)
    combined.drop('Loan_Amount_Term', axis=1, inplace=True)
    combined['Credit_History_Bad'] = combined['Credit_History'].map(lambda c: 1 if c==0 else 0)
    combined['Credit_History_Good'] = combined['Credit_History'].map(lambda c: 1 if c==1 else 0)
    combined['Credit_History_Unknown'] = combined['Credit_History'].map(lambda c: 1 if c==2 else 0)
    combined.drop('Credit_History', axis=1, inplace=True)
    property_dummies = pd.get_dummies(combined['Property_Area'], prefix='Property')
    combined = pd.concat([combined, property_dummies], axis=1)
    combined.drop('Property_Area', axis=1, inplace=True)
    #sc=MinMaxScaler()
    #combined['LoanAmount']=sc.fit_transform(combined[['LoanAmount']])
    #combined['Total_Income']=sc.fit_transform(combined[['Total_Income']])
    #combined['Debt_Income_Ratio']=sc.fit_transform(combined[['Debt_Income_Ratio']])
    return combined

In [12]:
Preprocess().isnull().sum()

Gender                    0
Married                   0
Education                 0
Self_Employed             0
LoanAmount                0
Singleton                 0
Small_Family              0
Large_Family              0
Total_Income              0
Debt_Income_Ratio         0
Very_Short_Term           0
Short_Term                0
Long_Term                 0
Very_Long_Term            0
Credit_History_Bad        0
Credit_History_Good       0
Credit_History_Unknown    0
Property_Rural            0
Property_Semiurban        0
Property_Urban            0
dtype: int64

In [13]:
def recover_train_test_target():
    global combined, data_train
    targets = data_train['Loan_Status'].map({'Y':1,'N':0})
    train = combined.head(614)
    New_data = combined.iloc[614:]
    return train, New_data, targets

In [14]:
train, New_data, targets = recover_train_test_target()
train.shape,targets.shape,New_data

((614, 20),
 (614,),
      Gender  Married  Education  Self_Employed  LoanAmount  Singleton  \
 614       1        1          1              0       110.0          0   
 615       1        1          1              0       126.0          1   
 616       1        1          1              0       208.0          0   
 617       1        1          1              0       100.0          0   
 618       1        0          0              0        78.0          0   
 ..      ...      ...        ...            ...         ...        ...   
 976       1        1          0              1       113.0          0   
 977       1        1          1              0       115.0          0   
 978       1        0          1              0       126.0          0   
 979       1        1          1              0       158.0          0   
 980       1        0          1              1        98.0          0   
 
      Small_Family  Large_Family  Total_Income  Debt_Income_Ratio  \
 614             0  

In [15]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(train,targets,test_size=0.2,random_state=25)

In [16]:
def RF():
    global New_data,data_test
    global x_train,x_test,y_train,y_test
    global new_x_train,new_x_test,new_data
    clf = RandomForestClassifier(n_estimators=50, max_features='sqrt')
    clf = clf.fit(x_train, y_train)
    features = pd.DataFrame()
    features['Feature'] = x_train.columns
    features['Importance'] = clf.feature_importances_
    features.sort_values(by=['Importance'], ascending=False, inplace=True)
    features.set_index('Feature', inplace=True)
    
    selector = SelectFromModel(clf, prefit=True)
    train_reduced = selector.transform(x_train)
    new_x_train=pd.DataFrame(train_reduced,columns=['Debt_Income_Ratio','Credit_History_Bad','Total_Income','LoanAmount','Credit_History_Good'])
    test_reduced = selector.transform(x_test)
    new_x_test=pd.DataFrame(test_reduced,columns=['Debt_Income_Ratio','Credit_History_Bad','Total_Income','LoanAmount','Credit_History_Good'])
    new_reduced=selector.transform(New_data)
    new_data=pd.DataFrame(new_reduced,columns=['Debt_Income_Ratio','Credit_History_Bad','Total_Income','LoanAmount','Credit_History_Good'])
    parameters = {'bootstrap': False,
              'min_samples_leaf': 3,
              'n_estimators': 50,
              'min_samples_split': 10,
              'max_features': 'sqrt',
              'max_depth': 6}

    rf = RandomForestClassifier(**parameters)
    rf.fit(new_x_train, y_train)
    pred=rf.predict(new_x_test)
    acc=accuracy_score(y_test,pred)
    cm=confusion_matrix(y_test,pred)
    CR=classification_report(y_test,pred)
    output = rf.predict(new_data).astype(int)
    df_output = pd.DataFrame()
    df_output['Loan_ID'] = data_test['Loan_ID']
    df_output['Loan_Predicted_Status'] = np.vectorize(lambda s: 'Y' if s==1 else 'N')(output)
    df_output[['Loan_ID','Loan_Predicted_Status']].to_csv('Loan_Approval_Prediction_submission@RF.csv',index=False)
    return features[:5],new_x_train
    print("Confusion Matrix:\n\n",cm)
    print("\nAccuracy Score:\n\n",str(np.round(acc*100,4))+' %')
    print("\nPredicted Values on the Test data:\n\n",pred)
    print("\nClassification Report:\n\n",CR)
    print("\nPredicted values on the New Data:\n\n",df_output)

In [17]:
RF()

(                     Importance
 Feature                        
 Total_Income           0.175310
 Debt_Income_Ratio      0.174400
 Credit_History_Bad     0.154347
 LoanAmount             0.150137
 Credit_History_Good    0.096198,
      Debt_Income_Ratio  Credit_History_Bad  Total_Income  LoanAmount  \
 0                111.0              5039.0     45.396396         0.0   
 1                144.0              5821.0     40.423611         0.0   
 2                200.0              6875.0     34.375000         0.0   
 3                126.0              6782.0     53.825397         0.0   
 4                120.0              6296.0     52.466667         0.0   
 ..                 ...                 ...           ...         ...   
 486               88.0              4192.0     47.636364         0.0   
 487              122.0              4732.0     38.786885         0.0   
 488              162.0             10180.0     62.839506         0.0   
 489              112.0              3

In [18]:
def LR():
    global New_data,data_test
    global x_train,x_test,y_train,y_test
    global new_x_train,new_x_test,new_data
    model=LogisticRegression()
    model.fit(new_x_train,y_train)
    pred=model.predict(new_x_test)
    acc=accuracy_score(y_test,pred)
    cm=confusion_matrix(y_test,pred)
    CR=classification_report(y_test,pred)
    output = model.predict(new_data).astype(int)
    df_output = pd.DataFrame()
    df_output['Loan_ID'] = data_test['Loan_ID']
    df_output['Loan_Predicted_Status'] = np.vectorize(lambda s: 'Y' if s==1 else 'N')(output)
    df_output[['Loan_ID','Loan_Predicted_Status']].to_csv('Loan_Approval_Prediction_submission@LR.csv',index=False)
    
    print("Confusion Matrix:\n\n",cm)
    print("\nAccuracy Score:\n\n",str(np.round(acc*100,4))+' %')
    print("\nPredicted Values on the Test data:\n\n",pred)
    print("\nClassification Report:\n\n",CR)
    print("\nPredicted values on the New Data:\n\n",df_output)

In [19]:
LR()

Confusion Matrix:

 [[21 22]
 [ 2 78]]

Accuracy Score:

 80.4878 %

Predicted Values on the Test data:

 [1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 0 1 0 1 1 1 0 0 1 1 1 1 0 1 0 1 1
 0 1 1 0 1 1 1 0 0 1 1 1 1 1 0 1 0 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1
 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 0 1 0 1 1
 1 1 1 1 1 1 1 1 1 1 1 1]

Classification Report:

               precision    recall  f1-score   support

           0       0.91      0.49      0.64        43
           1       0.78      0.97      0.87        80

    accuracy                           0.80       123
   macro avg       0.85      0.73      0.75       123
weighted avg       0.83      0.80      0.79       123


Predicted values on the New Data:

       Loan_ID Loan_Predicted_Status
0    LP001015                     Y
1    LP001022                     Y
2    LP001031                     Y
3    LP001035                     Y
4    LP001051                     Y
..        ...                   ...
3

In [20]:
def KNN():
    global New_data,data_test
    global x_train,x_test,y_train,y_test
    global new_x_train,new_x_test,new_data
    model=KNeighborsClassifier()
    model.fit(new_x_train,y_train)
    pred=model.predict(new_x_test)
    acc=accuracy_score(y_test,pred)
    cm=confusion_matrix(y_test,pred)
    CR=classification_report(y_test,pred)
    output = model.predict(new_data).astype(int)
    df_output = pd.DataFrame()
    df_output['Loan_ID'] = data_test['Loan_ID']
    df_output['Loan_Predicted_Status'] = np.vectorize(lambda s: 'Y' if s==1 else 'N')(output)
    df_output[['Loan_ID','Loan_Predicted_Status']].to_csv('Loan_Approval_Prediction_submission@knn.csv',index=False)
    
    print("Confusion Matrix:\n\n",cm)
    print("\nAccuracy Score:\n\n",str(np.round(acc*100,4))+' %')
    print("\nPredicted Values on the Test data:\n\n",pred)
    print("\nClassification Report:\n\n",CR)
    print("\nPredicted values on the New Data:\n\n",df_output)

In [21]:
KNN()

Confusion Matrix:

 [[ 5 38]
 [12 68]]

Accuracy Score:

 59.3496 %

Predicted Values on the Test data:

 [1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1
 1 1 0 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 0 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 1 1 1 1]

Classification Report:

               precision    recall  f1-score   support

           0       0.29      0.12      0.17        43
           1       0.64      0.85      0.73        80

    accuracy                           0.59       123
   macro avg       0.47      0.48      0.45       123
weighted avg       0.52      0.59      0.53       123


Predicted values on the New Data:

       Loan_ID Loan_Predicted_Status
0    LP001015                     Y
1    LP001022                     Y
2    LP001031                     Y
3    LP001035                     N
4    LP001051                     Y
..        ...                   ...
3

In [22]:
def SVM():
    global New_data,data_test
    global x_train,x_test,y_train,y_test
    global new_x_train,new_x_test,new_data
    model=SVC()
    model.fit(new_x_train,y_train)
    pred=model.predict(new_x_test)
    acc=accuracy_score(y_test,pred)
    cm=confusion_matrix(y_test,pred)
    CR=classification_report(y_test,pred)
    output = model.predict(new_data).astype(int)
    df_output = pd.DataFrame()
    df_output['Loan_ID'] = data_test['Loan_ID']
    df_output['Loan_Predicted_Status'] = np.vectorize(lambda s: 'Y' if s==1 else 'N')(output)
    df_output[['Loan_ID','Loan_Predicted_Status']].to_csv('Loan_Approval_Prediction_submission@svm.csv',index=False)
    
    print("Confusion Matrix:\n\n",cm)
    print("\nAccuracy Score:\n\n",str(np.round(acc*100,4))+' %')
    print("\nPredicted Values on the Test data:\n\n",pred)
    print("\nClassification Report:\n\n",CR)
    print("\nPredicted values on the New Data:\n\n",df_output)

In [23]:
SVM()

Confusion Matrix:

 [[ 0 43]
 [ 0 80]]

Accuracy Score:

 65.0407 %

Predicted Values on the Test data:

 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1]

Classification Report:

               precision    recall  f1-score   support

           0       0.00      0.00      0.00        43
           1       0.65      1.00      0.79        80

    accuracy                           0.65       123
   macro avg       0.33      0.50      0.39       123
weighted avg       0.42      0.65      0.51       123


Predicted values on the New Data:

       Loan_ID Loan_Predicted_Status
0    LP001015                     Y
1    LP001022                     Y
2    LP001031                     Y
3    LP001035                     N
4    LP001051                     Y
..        ...                   ...
3

In [24]:
def DT():
    global New_data,data_test
    global x_train,x_test,y_train,y_test
    global new_x_train,new_x_test,new_data
    model=DecisionTreeClassifier()
    model.fit(new_x_train,y_train)
    pred=model.predict(new_x_test)
    acc=accuracy_score(y_test,pred)
    cm=confusion_matrix(y_test,pred)
    CR=classification_report(y_test,pred)
    output = model.predict(new_data).astype(int)
    df_output = pd.DataFrame()
    df_output['Loan_ID'] = data_test['Loan_ID']
    df_output['Loan_Predicted_Status'] = np.vectorize(lambda s: 'Y' if s==1 else 'N')(output)
    df_output[['Loan_ID','Loan_Predicted_Status']].to_csv('Loan_Approval_Prediction_submission@DT.csv',index=False)
    
    print("Confusion Matrix:\n\n",cm)
    print("\nAccuracy Score:\n\n",str(np.round(acc*100,4))+' %')
    print("\nPredicted Values on the Test data:\n\n",pred)
    print("\nClassification Report:\n\n",CR)
    print("\nPredicted values on the New Data:\n\n",df_output)

In [25]:
DT()

Confusion Matrix:

 [[22 21]
 [13 67]]

Accuracy Score:

 72.3577 %

Predicted Values on the Test data:

 [1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 0 1 0 1 1 1 0 0 1 1 1 0 0 1 0 1 0
 0 1 0 0 1 1 1 0 0 1 1 0 1 0 0 1 0 0 0 1 1 1 1 0 1 1 1 0 1 1 1 1 1 0 1 1 1
 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 0 1 1 1 1 0 1 0 0 1 1 1
 1 0 0 0 1 1 1 1 1 1 1 1]

Classification Report:

               precision    recall  f1-score   support

           0       0.63      0.51      0.56        43
           1       0.76      0.84      0.80        80

    accuracy                           0.72       123
   macro avg       0.69      0.67      0.68       123
weighted avg       0.71      0.72      0.72       123


Predicted values on the New Data:

       Loan_ID Loan_Predicted_Status
0    LP001015                     Y
1    LP001022                     Y
2    LP001031                     Y
3    LP001035                     N
4    LP001051                     Y
..        ...                   ...
3

In [26]:
list=[[20,60,41,37,65]]
sc=MinMaxScaler()
sc.fit_transform(list)

array([[0., 0., 0., 0., 0.]])